In [2]:
import pandas as pd
# Note: we are completely working with closing at the mid price

In [3]:
# --- 0.

# Read CSV from a relative folder
df = pd.read_csv('/home/vishi/bolinger/15-7-25-nifty-bb.csv')

rolling_period = 20
# Calculate rolling mean and std (interval 20, excluding current row)
df['mid'] = df['close'].shift(1).rolling(window=rolling_period).mean()
df['SD'] = df['close'].shift(1).rolling(window=rolling_period).std()

# Construct Bollinger Bands
K = 2
df['upper'] = df['mid'] + (K * df['SD'])
df['lower'] = df['mid'] - (K * df['SD'])

df['time'] = range(len(df))

# Save to new CSV
df.to_csv('/home/vishi/bolinger/15-7-25-nifty-bb.csv', index=False)

In [ ]:
# --- 1.
df = pd.read_csv('../Datasets/1_SENSEX_with_rolling.csv')

df = df.dropna(subset=['mid', 'SD', 'upper', 'lower']).reset_index(drop=True)

def get_position(row):
    if row['close'] > row['upper']:
        return 2
    elif row['upper'] >= row['close'] > row['mid']:
        return 1
    elif row['mid'] >= row['close'] > row['lower']:
        return -1
    elif row['close'] <= row['lower']:
        return -2  
    else:
        return None  # In case of missing data

# 2 -> closed price is above upper band
# 1 -> closed price is between upper band and middle band
# -1 -> closed price is between middle band and lower band
# -2 -> closed price is below lower band

df['stock_position'] = df.apply(get_position, axis=1)

# Save to a new CSV
csv_file = '../Datasets/2_SENSEX_ClassificationFalse.csv'
df.to_csv(csv_file, index=False)

In [67]:
# --- 2.
df = pd.read_csv(csv_file)

# Create a column for the previous position
df['prev_position'] = df['stock_position'].shift(1)

def sq_off_position(row):
    prev = row['prev_position']
    curr = row['stock_position']
    if prev == 1 and curr == -1:
        return -1  # Crossed mid band from above
    elif prev == -1 and curr == 1:
        return 1  # Crossed mid band from below
    elif prev > -2 and curr == -2:
        return -2  # Crossed lower band from above
    elif prev < 2 and curr == 2:
        return 2  # Crossed upper band from below
    else:
        return 0  # Nothing

# 0 -> nothing
# -1 -> Closed price crosses the mid band from above
# 1 -> Closed price crosses the mid band from below 
# -2 -> Closed price crosses the lower band from above 
# 2 -> Closed price crosses the upper band from below 

df['sq_off_position'] = df.apply(sq_off_position, axis=1)
df.drop(columns=['prev_position'], inplace=True)

# Save the updated DataFrame to the same CSV file
df.to_csv(csv_file, index = False)

In [68]:
# --- 3.
df = pd.read_csv(csv_file)

profit_col = []
close_time_col = [] # --- 5.
n = len(df)
for i in range(n):
    profit = 0
    close_time = -1
    if df.loc[i, 'stock_position'] == 2:
        entry_price = df.loc[i, 'close']
        # Find next sq_off_position == -1 or -2 (whichever comes first)
        mask = (df.loc[(i+1):, 'sq_off_position'].isin([-1, -2]))
        close_idxs = df.index[(i+1):][mask]
        if not close_idxs.empty:
            exit_idx = close_idxs[0]
            exit_price = df.loc[exit_idx, 'close']
            profit = (entry_price - exit_price) / entry_price * 100  # Short profit %
            close_time = df.loc[exit_idx, 'time']
    elif df.loc[i, 'stock_position'] == -2:
        entry_price = df.loc[i, 'close']
        # Find next sq_off_position == 1 or 2 (whichever comes first)
        mask = (df.loc[(i+1):, 'sq_off_position'].isin([1, 2]))
        close_idxs = df.index[(i+1):][mask]
        if not close_idxs.empty:
            exit_idx = close_idxs[0]
            exit_price = df.loc[exit_idx, 'close']
            profit = (exit_price - entry_price) / entry_price * 100  # Long profit %
            close_time = df.loc[exit_idx, 'time']
    profit_col.append(profit)
    close_time_col.append(close_time)

df['expected_profit'] = profit_col
df['close_time'] = close_time_col

# Save the updated DataFrame to the same CSV file
df.to_csv(csv_file, index=False)

In [69]:
# --- 4
df = pd.read_csv(csv_file)

def get_label(profit):
    if profit > 0:
        return 1
    elif profit < 0:
        return 0
    else:
        return -1

df['label'] = df['expected_profit'].apply(get_label)

df.to_csv(csv_file, index=False)